In [ ]:
# Custom methods to clean up the source code a bit
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

class cust():

    def __init__(self, dat):
        self.dat = dat


    def xy(self, x_dummies_list, X_list, y_list):

        X = self.dat[X_list]
        y = self.dat[y_list]

        # One hot encoding the pg column
        X = pd.get_dummies(X, columns = x_dummies_list, drop_first = True)
        # C_cols should be after get_dummies so we get ALL columns
        x_cols = X.columns
        # Convert the two into value arrays
        X = X.values
        y = y.values
        # We need y as a 1D array
        y = np.ravel(y)

        return X,y,x_cols


    # Same as xy except returns a dataframe instead of a float64 np array
    def xy_df(self, x_dummies_list, X_list, y_list):

        X = self.dat[X_list]
        y = self.dat[y_list]

        # One hot encoding the pg column
        X = pd.get_dummies(X, columns = x_dummies_list, drop_first = True)
        # C_cols should be after get_dummies so we get ALL columns
        x_cols = X.columns
        # We need y as a 1D array
        y = np.ravel(y)

        return X,y,x_cols


    def clean_data(self):
        self.dat.replace([np.inf, -np.inf], np.nan) # Replace inf
        self.dat = self.dat.dropna(axis=0, how = 'any') # Drop NA's on the rows axis
        # I kept getting a value error and this was the only thing that seemed to fix it
        self.dat = self.dat[~self.dat.isin([np.nan, np.inf, -np.inf]).any(1)]
        return self.dat


    def outlier_removal(self,var):
        IQR = self.dat[var].describe()['75%'] - self.dat[var].describe()['25%']
        min_val = self.dat[var].describe()['25%'] - (IQR * 1.5)
        max_val = self.dat[var].describe()['75%'] + (IQR * 1.5)

        self.dat = self.dat[(self.dat[var] > min_val) & (self.dat[var] < max_val)]
        plt.boxplot(self.dat[var])
        return self.dat

    @staticmethod
    def comparison_df(y_pred, y_test):
        # Dataframe of pred and actual y
        comparison_df = pd.DataFrame({'y_pred':y_pred, 'y_test':y_test})
        comparison_df['abs_difference'] = abs( comparison_df['y_pred'] - comparison_df['y_test'] )
        comparison_df['real_difference'] = comparison_df['y_pred'] - comparison_df['y_test']
        print(comparison_df.describe())
        # Show all sums
        print(comparison_df.sum())
        # Show average difference
        print ("Average Difference: ", comparison_df.sum()[2] / len(comparison_df))

        return comparison_df

In [ ]:
#%cd /content/drive/MyDrive/Prediction/Rangpur_Region

/content/drive/MyDrive/Prediction/Rangpur_Region


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
columns = ['Time', 'Node', 'Total(MB)']
data = pd.read_excel("Prediction_dataset.xlsx")
data.Time = pd.to_datetime(data.Time)
node_list = data["Node"]
nodes = node_list.unique()

In [ ]:
from kats.consts import TimeSeriesData
import pandas as pd
import LSTM_Prep_univariate
import numpy as np
from sklearn.metrics import mean_squared_error
from kats.models.prophet import ProphetModel, ProphetParams
import matplotlib.pyplot as plt

# Specify parameters
params = ProphetParams(seasonality_mode="multiplicative")

split = 1
sequence_length = 8
pred_days = 30 # predicted_vs_actual plot
EPOCHS = 50
validation = 0.2
import csv
f = open('excel_out_KATS_kpi_.csv', 'w')
g = open('excel_out_KATS_.csv', 'w')
writer = csv.writer(f)
writer2 = csv.writer(g)
header_written = False

forecast_length = 30

for node in nodes:
    print(node)
    node_index = data["Node"] == node
    my_data = data[node_index]
    row_data=[node]

    my_data = my_data.set_index("Time")
    python = my_data["Total(GB)"].to_frame()
    python = python.sort_index(ascending=True)
    #print(python)

    # Split data into train and test set
    train = python.iloc[:-pred_days]
    test = python.iloc[-pred_days:]
    #print(test)

    ts = TimeSeriesData(train.reset_index(), time_col_name="Time")

    ts.plot(cols=["Total(GB)"])

    # Create a model instance
    m = ProphetModel(ts, params)

    # Fit mode
    m.fit()

    # Forecast
    fcst = m.predict(steps=30, freq="D")
    #print(fcst)

    m.plot()
    plt.plot(python["Total(GB)"])
    test = test.values
    train = train.values
    predictions = fcst.fcst

    rmse = np.sqrt(mean_squared_error(test, predictions))
    print('Test RMSE: %.3f' % rmse)

    meas_nam, meas_v = calc_metrics(test,predictions)
    if not header_written:
        writer.writerow(['eNodeB']+meas_nam)
        header_written=True
    writer.writerow(row_data+meas_v)
    gap2 = [''] * (len(train)+1)
    writer2.writerow(row_data+train.tolist()+test.tolist())
    writer2.writerow(gap2+predictions.values.tolist())
f.close()
g.close()

In [ ]:
!pip install kats

     |████████████████████████████████| 6.3 MB 8.5 MB/s 
     |████████████████████████████████| 866 kB 44.9 MB/s 
     |████████████████████████████████| 64 kB 2.9 MB/s 
     |████████████████████████████████| 503 kB 47.6 MB/s 
     |████████████████████████████████| 3.3 MB 35.7 MB/s 
     |████████████████████████████████| 23.9 MB 14 kB/s 
     |████████████████████████████████| 9.8 MB 41.4 MB/s 
     |████████████████████████████████| 26.3 MB 92 kB/s 
     |████████████████████████████████| 486 kB 69.8 MB/s 
  Created wheel for kats: filename=kats-0.1.0-py3-none-any.whl size=286607 sha256=b2e5d93114bfda287e848afad037e3155cf2bea2a910ff4f00b506c49494bf8e
  Stored in directory: /root/.cache/pip/wheels/c8/dd/5b/cc7cb7fc37c5b838c65e504437bba4c1828a3fbb473c0c11be
  Created wheel for fbprophet: filename=fbprophet-0.7-py3-none-any.whl size=6637200 sha256=2423045ed29adb88890903b6ea643fce36adbf0ca32e09abf41b025c6c71575a
  Stored in directory: /root/.cache/pip/wheels/82/e8/a8/53f37f0a409bc51f8

In [ ]:
def calc_metrics(true_y, predicted_y):
    metric_names = ['Forecast Bias', 'R-sqr', 'MAE', 'RMSE', 'MAPE']

    forecast_errors = [predicted_y[i]-true_y[i] for i in range(len(predicted_y))]
    bias = sum(forecast_errors) * 1.0/len(predicted_y)
    mean_absolute_error = np.mean( np.abs(forecast_errors) )
    root_mean_squared_error = np.sqrt(np.mean([number ** 2 for number in forecast_errors]))
    Unexplained_var = np.sum([number ** 2 for number in forecast_errors])
    mean_of_true = np.mean(true_y)
    Total_var = np.sum([(number-mean_of_true) ** 2 for number in true_y])
    r_sqr = 1 - Unexplained_var/Total_var
    percentage_error = [(predicted_y[i]-true_y[i])/true_y[i] if true_y[i]>0 else 0 for i in range(len(predicted_y))]
    mean_absolute_percentage_error = np.mean( np.abs(percentage_error) ) * 100

    metric_values = [bias, r_sqr, mean_absolute_error, root_mean_squared_error, mean_absolute_percentage_error]

    return metric_names, metric_values